---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [5]:
def graph_identification():
    
    # Your Code Here
    import matplotlib.pyplot as plt
    
    algos_list = []
    for graph in P1_Graphs:
        degrees = graph.degree()
        degree_values = sorted(set(degrees.values()))
        histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(graph)) for i in degree_values]
        clustering = nx.average_clustering(graph)
        
        if len(histogram) > 10:
            algos_list.append('PA')
        elif clustering < 0.1:
            algos_list.append('SW_H')
        else:
            algos_list.append('SW_L')
            
        #to help visualize, but delete/comment out plot lines pior to submitting to autograder
#         plt.plot(degree_values, histogram, 'o')
#         plt.xlabel('Degree')
#         plt.ylabel('Fraction of Nodes')
#         plt.xscale('log')
#         plt.yscale('log')
#         plt.show()
        
    return algos_list # Your Answer Here
# graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [3]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))
# G.nodes(data=True) # this line was added by me - a student - to visualize nodes in G

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [32]:
def salary_predictions():
    
    # Your Code Here
    from sklearn.linear_model import LogisticRegression # seems to run fastest, still gets full marks with autograder
    from sklearn.ensemble import RandomForestClassifier # seems to yield highest accuracy but results are only 1 signficant digits, not sure why

    df = pd.DataFrame(index=G.nodes())
    df['department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['salary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())

    # Manual train, test, split. Very unpythonic but it works.
    df_train = df.dropna()
    X_train = df_train.drop(['salary'], axis=1)
    y_train = df_train['salary']
    df_test = df[df.isnull().any(axis=1)]
    X_test = df_test.loc[:, df.columns != 'salary']
    y_test = df_test['salary'] # I guess this is not needed as predict_proba will generate a y_test column

    clf = LogisticRegression()
    # clf = GradientBoostingClassifier()
    clf.fit(X_train, y_train)
    y_test = clf.predict_proba(X_test)[:,1]
    answer = pd.Series(data=y_test, index=X_test.index)

    return answer # Your Answer Here
# salary_predictions()

1       0.154604
2       0.586857
5       0.961224
8       0.133419
14      0.355656
18      0.185911
27      0.276013
30      0.330870
31      0.193383
34      0.136541
37      0.126426
40      0.249572
45      0.181345
54      0.254090
55      0.201374
60      0.216587
62      0.994823
65      0.447253
77      0.074921
79      0.129034
97      0.103141
101     0.084137
103     0.196728
108     0.129447
113     0.473533
122     0.067397
141     0.429334
142     0.910960
144     0.092426
145     0.423334
          ...   
913     0.069376
914     0.091510
915     0.016219
918     0.116729
923     0.054798
926     0.089405
931     0.074059
934     0.040456
939     0.028103
944     0.018593
945     0.052710
947     0.102571
950     0.155215
951     0.061867
953     0.049030
959     0.014974
962     0.016506
963     0.154702
968     0.089265
969     0.082809
974     0.067699
984     0.038891
987     0.097024
989     0.082886
991     0.106629
992     0.016235
994     0.018175
996     0.0167

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [4]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [5]:
def new_connections_predictions():
    
    # Your Code Here
    from sklearn.linear_model import LogisticRegression # results in AUC ~ 0.83 :( so will try GradientBoosting for next submission
    from sklearn.ensemble import GradientBoostingClassifier 
        # Results in AUC of ~ 0.83 but slightly higher than LogisticRegression, 
        # throwing in the towel, maybe common neighbors will yield a higher AUC
    
    # This list comprehension was derived from lecture code for analyzing cities
    future_connections['PA_scores'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
#     print(future_connections.head(20))

    # Manual train, test, split. Very unpythonic but it works.
    df_train = future_connections.dropna()
    X_train = df_train.drop(['Future Connection'], axis=1)
    y_train = df_train['Future Connection']
    df_test = future_connections[future_connections.isnull().any(axis=1)]
    X_test = df_test.loc[:, df_test.columns != 'Future Connection']
    y_test = df_test['Future Connection'] # I guess this is not needed as predict_proba will generate a y_test column

#     clf = LogisticRegression()
    clf = GradientBoostingClassifier()
    clf.fit(X_train, y_train)
    y_test = clf.predict_proba(X_test)[:,1] # predict_proba yields tuples for probababilty of both 0 and 1, so 1 is selected with [:,1]
    answer = pd.Series(data=y_test, index=X_test.index)
    return answer # Your Answer Here
# new_connections_predictions()

(107, 348)    0.059621
(542, 751)    0.021144
(20, 426)     0.383380
(50, 989)     0.016581
(942, 986)    0.014627
(324, 857)    0.020207
(13, 710)     0.314956
(19, 271)     0.475153
(319, 878)    0.016002
(659, 707)    0.021144
(49, 843)     0.015290
(208, 893)    0.019870
(377, 469)    0.193261
(405, 999)    0.021144
(129, 740)    0.047581
(292, 618)    0.048328
(239, 689)    0.014938
(359, 373)    0.094795
(53, 523)     0.056537
(276, 984)    0.012333
(202, 997)    0.014529
(604, 619)    0.016581
(270, 911)    0.014938
(261, 481)    0.161418
(200, 450)    0.193261
(213, 634)    0.022104
(644, 735)    0.021144
(346, 553)    0.035563
(521, 738)    0.048328
(422, 953)    0.054691
                ...   
(672, 848)    0.014938
(28, 127)     0.212475
(202, 661)    0.036832
(54, 195)     0.125506
(295, 864)    0.019606
(814, 936)    0.021144
(839, 874)    0.014627
(139, 843)    0.019606
(461, 544)    0.057821
(68, 487)     0.057821
(622, 932)    0.019870
(504, 936)    0.073667
(479, 528) 